In [ ]:
def pdf_sim(embed, tab_old, result=None, max_iter=1, r_state = 2):
    """
    Wanted statistics?: number of words, similarity, number of pages, percentage of words exactly the same,
    in = table
    """ 
    # for each random row in table, replace=False all rows only sampled once
    sample = tab_old.sample(n=max_iter, replace=False, random_state=r_state)
    not_sample = tab_old.drop(sample.index)
    
    #download two papers from table
    loc_name_Arxiv = 'pdf sim/Arxiv.pdf'
    loc_name_Anth = 'pdf sim/Anth.pdf'
    
    
    sample[['Succes', 'w_Anth', 'w_Arxiv', 'w_both', 'pages_Anth', 'pages_Arxiv', 'cosine', 'len_blocks', 'len_Anth', 'len_Arxiv', 'len_both', 'ref_Anth', 'ref_Arxiv', 'Jaccard' ]] =np.nan
    
    
    for index,series in sample.iterrows():
        sample.loc[[index],['Succes']]=0


        #open paper in jupiter/blocks
        try: #sometimes download not available. Then download works but does not give a good pdf so reading does not work
            urllib.request.urlretrieve(series['pdf_url_Arxiv'], loc_name_Arxiv)
            urllib.request.urlretrieve(series['url']+'.pdf', loc_name_Anth)
            print('start with two papers')
            doc_Arxiv = fitz.open(loc_name_Arxiv)
            doc_Anth = fitz.open(loc_name_Anth)

            b_Anth = get_blocks(doc_Anth)
            b_Arxiv = get_blocks(doc_Arxiv)
            
            #try removing sidebar from arxiv
            # if we DO find something
            if [i for i, x in enumerate(b_Arxiv) if x.find('arXiv:')>= 0] !=[]:
                i_sidebar = [i for i, x in enumerate(b_Arxiv) if x.find('arXiv:')>= 0][0]
                if len(b_Arxiv[i_sidebar])< 50: #dont remove references by accident normal is length 41
                    x = b_Arxiv.pop(i_sidebar)
                
#             print('remove_bar skipped')
#             remove_bar = doc_Arxiv.load_page(0).get_text("text",clip=rect)
#             try:
#                 b_Arxiv.remove(remove_bar)
#                 print('bar removed')
#             except:
#                 print('Unable to remove bar from arxiv')
            
#             elem_both_list = set()
            elem_both_list = set(b_Anth)&set(b_Arxiv)
            #sometimes same occurs twice in paper then remove this from elem_both_list
            dup = [x for x in elem_both_list if b_Anth.count(x)>1]
            dup = set(dup + [x for x in elem_both_list if b_Anth.count(x)>1])
            for el in dup:
                elem_both_list.remove(el)
                
        
            #number of citations
            if [i for i, x in enumerate(b_Anth) if x.find('References')>= 0] ==[]:
                ref_Anth='NF'
                ref_Arxiv='NF'
            elif [i for i, x in enumerate(b_Anth) if x.find('References')>= 0] ==[]:
                ref_Anth='NF'
                ref_Arxiv='NF'
            else:
                start_ref_Anth = [i for i, x in enumerate(b_Anth) if x.find('References')>= 0][0]
                
                start_ref_Arxiv = [i for i, x in enumerate(b_Arxiv) if x.find('References')>= 0][0]
                
#                 ref_Anth = max(sum([x.count('[') for x in b_Anth[start_ref_Anth:]]),len(b_Anth[start_ref_Anth:]))
#                 ref_Arxiv = max(sum([x.count('[') for x in b_Anth[start_ref_Arxiv:]]),len(b_Anth[start_ref_Arxiv:]))
                
                ref_Anth,ref_Arxiv = max([len(b_Anth[start_ref_Anth:]),len(b_Arxiv[start_ref_Arxiv:])],
                                         [sum([x.count('[') for x in b_Anth[start_ref_Anth:]]),
                                          sum([x.count('[') for x in b_Arxiv[start_ref_Arxiv:]])])
                
            
            #join blocks
            p_Anth = join_blocks(b_Anth, elem_both_list)
            p_Arxiv = join_blocks(b_Arxiv, elem_both_list)
    
            #similarity
            weight_avg, w_Anth, w_Arxiv, len_blocks, len_Anth, len_Arxiv, jacc = sim(p_Anth,p_Arxiv,embedding=embed)
            
            # add statistics to table 'sample'
            sample.loc[[index],['cosine', 'w_Anth', 'w_Arxiv', 'len_blocks', 'len_Anth', 'len_Arxiv', 'ref_Anth', 'ref_Arxiv', 'Jaccard']]= weight_avg, w_Anth, w_Arxiv, len_blocks, len_Anth, len_Arxiv, ref_Anth, ref_Arxiv, jacc
            sample.loc[[index],['w_both','len_both','pages_Anth', 'pages_Arxiv']] = [len(clean_n("".join(elem_both_list)).split()),len(elem_both_list),doc_Anth.page_count, doc_Arxiv.page_count]
            
            sample.loc[[index],['Succes']]=1
            
#             #remove papers
#             if os.path.exists("pdf sim/Anth.pdf"):
#                   os.remove("pdf sim/Anth.pdf")
#             else:
#                 print("The anthology file does not exist") 
#             if os.path.exists("pdf sim/Arxiv.pdf"):
#                   os.remove("pdf sim/Arxiv.pdf")
#             else:
#                 print("The arxiv file does not exist") 

        except: #error with downloading 
            print('Exception found some error')
            next 
              
        time.sleep(3)
        
    #end loop
    # join tables
    if type(result) ==pd.DataFrame:
        sample = sample.merge(result,how='outer')
    return not_sample, sample
    
    
    
def get_blocks(doc):
    blocks = []
    for page in doc.pages():
        b_p = page.get_text('blocks')
        for b in b_p:
            if b[6] == 0: #only text not metadata pictures.
                blocks.append(b[4])
                
    return blocks

def join_blocks(blocks, elem_both_list):
    """Joins same and diff paragraphs until it same comes after difference 
    then add to list and start new paragraph with the shared block"""
    parag = []
    #negative so diff before first same get added to first paragraph
    is_diff = -100 #dummy variable to know when to append paragraph. Join same + diff, if same again start new paragraph
    join = '' #join outside of loop
    for b in blocks:
        if b == ' ':
            next
        if b in elem_both_list: # if block is same
            if is_diff >= 1: #last block was different, start new paragraph
                parag.append(join)
                is_diff = 0
                join = b
            elif is_diff == 0: #same followed by same, new paragraph and see if diff follows
                # is_diff stays 0
                parag.append(join)
                join = b
            else: #  first block of same, not new paragraph 
                is_diff = 0
                join+=b
#                 join+=' '
                # is_diff is 0 and stays 0
        else: #block is different to the other paper
            is_diff+= 1 
            join+=' '
            join+=b
            
    #add last paragraph to list named parag since it has not been added
    parag.append(join)
            
    return parag

def join_diff(blocks, elem_both_list):
    """Only join different blocks not same ones"""
    parag = []
    is_diff = 0 #dummy variable to know when to append paragraph. Join same + diff, if same again start new paragraph
    join = ''
    for b in blocks:
        if b in elem_both_list: # same: if not empty add, else go to next
            if is_diff==1:
                parag.append(join)
                join = ''
                is_diff=0
            else:next               
        else: # not in elem list
            join+=b #add element to join string
            is_diff=1
    parag.append(join)
            
    return parag
    
def sim(anth, arxiv, embedding):
#     print('start sim')
#     print('Length Anthology: {}\nLength Arxiv:{}'.format(len(anth),len(arxiv)))
    len_Anth = len(anth)
    len_Arxiv = len(arxiv)
    
    set_Anth = set(clean_n(" ".join(anth)).split())
    set_Arxiv = set(clean_n(" ".join(arxiv)).split())
    
    # jacc = Intersection/Union
    jacc = len(set_Anth & set_Arxiv)/len(set_Anth | set_Arxiv)
    
    
    #action if not same length make it one big string
    if len(anth) != len(arxiv):
        u_anth = " ".join(anth)
        u_arxiv = " ".join(arxiv)
        #could be just length of words??
#         print('Updated length Anthology: {}\nUpdated length Arxiv:{}'.format(len(u_anth),len(u_arxiv)))
        df = pd.DataFrame()
        df['Anth'] = [u_anth]
        df['Arxiv'] = [u_arxiv]
        len_blocks = 1
    else:
        df = pd.DataFrame()
        df['Anth'] = anth
        df['Arxiv'] = arxiv
        len_blocks= len(anth)

    df['Anth'] = df['Anth'].apply(clean_n)
    df['Arxiv'] = df['Arxiv'].apply(clean_n)
#     print(df)
    
    df['words Anth'] = df['Anth'].apply(lambda x: len(x.split()))
    df['words Arxiv'] = df['Arxiv'].apply(lambda x: len(x.split()))
#     print('start cosin')
    df['cosine'] = df.apply(lambda x:cosine_similarity(embedding([x.Anth]),embedding([x.Arxiv])), axis=1)
    df['cosine'] = df['cosine'].astype(float)

    #nr of words per paper
    w_Anth = sum(df['words Anth'])
    w_Arxiv = sum(df['words Arxiv'])
    
    df['weight'] = df['words Anth']+df['words Arxiv']
    
    df2 = df[df['words Anth']>50] #shorter pieces are often just headings, 
    
    weight_avg = round(np.average( df2['cosine'], weights = df2['weight']),6)
        
#     return  df, weight_avg #possible to evaluate to blocks and weighted average.
#     print(weight_avg, w_Anth, w_Arxiv, len_blocks, len_Anth, len_Arxiv)
    
    return weight_avg, w_Anth, w_Arxiv, len_blocks, len_Anth, len_Arxiv, jacc  # or return sim score and len words and diff words.
